In [4]:
# to be able to run this script as module
import sys
import os
import pandas as pd
a=2

module_path = os.path.abspath(os.getcwd())    

if module_path not in sys.path:       
    sys.path.append(module_path)

In [21]:
def ShowTypes(X):
    
    #train.dtypes
    columns = list(X)

    print ('\033[1m', "Categorical variables of the dataset are: ")
    print ('\033[0m')

    categorical_variables = X.select_dtypes(include = 'object').columns.tolist()
    print (* categorical_variables, sep = "\n")
            

    print (' ')
    print ('\033[1m', "Numerical variables of the dataset are: ")
    print ('\033[0m')

    numeric = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_variables = X.select_dtypes(include = numeric).columns.tolist()
    
    print(* numeric_variables, sep = "\n")
    
    return

def ShowLevels(X):
    
    categorical_variables = X.select_dtypes(include = 'object').columns.tolist()
    
    print("Number of records in the dataset is ",  X.shape[0]) 
    print ('='*50)
    print(' ')
    for variable in categorical_variables:
        print ("Number of ", variable, " levels: ", X[variable].nunique()) 
    return


In [22]:
def get_title(name):
    
    import re as re

    title_search = re.search(' ([A-Za-z]+)\.', name)
    
    if title_search:
        return title_search.group(1)
    return 

In [20]:
def get_last_name(name):
    
    last_name = name.split(',')[0]
    
    if last_name:
        return last_name
    
    return

def get_other_full_name(name):
    
    other_full_name = name[name.find("(")+1 : name.find(")")]
    
    if name.find("(") != -1:
        return other_full_name
    return

def get_other_last_name(name):
    
    other_full_name = name[name.find("(")+1 : name.find(")")]
    
    other_last_name = other_full_name.split(' ')[-1]
    
    if name.find("(") != -1:
        return other_last_name
    return    

def get_capital_letters(word):
    
    import re as re
    capital_letters = re.search('[A-Z]+', word)
    
    if capital_letters:
        return capital_letters.group(0)
    return

In [20]:
def impute_most_frequent (X, y):
    
    import pandas as pd
    
    X_out = X
    most_frequent = pd.get_dummies(X_out[y]).sum().sort_values(ascending=False).index[0]
    X_out[y] = X_out[y].fillna(most_frequent)
    
    return X_out

def impute_mean (X, y):
    
    import pandas as pd
    from sklearn.preprocessing import Imputer
    
    X_out = X
    mean_imputer = Imputer (missing_values = 'NaN', strategy = 'mean', axis = 0)
    imputed_y = mean_imputer.fit_transform (X_out[[y]])
    X_out[y] = imputed_y
    
    return X_out

In [40]:
# def ffff(learning_rate_start,learning_rate_end,
#             *n_estimators_start, n_estimators_end,
#             max_depth_start, max_depth_end,
#             min_samples_split_start, min_samples_split_end,
#             min_samples_leaf_start, min_samples_leaf_end):
    
#     import numpy as np
#     learning_rates = [float(x) for x in np.linspace(
#         start = learning_rate_start,
#         stop = learning_rate_end, 
#         num = 50)]

#     n_estimators = [int(x) for x in np.linspace(
#         start = n_estimators_start,
#         stop = n_estimators_end,
#         num = 50)]

#     max_depth = [int(x) for x in np.linspace(
#         start = max_depth_start, 
#         stop = max_depth_end, 
#         num = 50)]

#     max_depth.append(None)

#     min_samples_split = [int(x) for x in np.linspace(
#         start = min_samples_split_start,
#         stop = min_samples_split_end, 
#         num = 10)]

#     min_samples_leaf = [int(x) for x in np.linspace(
#         start = min_samples_leaf_start, 
#         stop = min_samples_leaf_end, 
#         num = 50)]
    
#     return learning_rates

# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'learning_rate': learning_rates}



In [28]:
def impute_regressor(data, variable):
    
    from sklearn.ensemble import RandomForestRegressor

    #data.columns.tolist()
    all_columns = ['Pclass', 'Sex', 'Age', 'SibSp','Parch',
                         'FamilySize','Fare','Embarked','IsAlone','Title' ]
    
    data_for_var = data[all_columns]
    
    columns_to_check = all_columns
    columns_to_check.remove(variable) # REMOVES IT FROM ALL COLUMNS TOO!!!!
    
    for column in columns_to_check:
        if data_for_var[column].isna().sum() != 0:
            data_for_var.drop (column, 1, inplace = True)

    categorical_variables = data_for_var.select_dtypes(include = 'object').columns.tolist()
    dummy_variables = pd.get_dummies(data_for_var.loc[:,categorical_variables])

    data_for_var = pd.merge(data_for_var, dummy_variables, left_index = True, right_index = True)
    data_for_var = data_for_var.drop(categorical_variables, axis = 1)


    X_train = data_for_var.loc[data_for_var[variable].isna()==False, :]
    X_train = X_train.drop (variable, 1)

    y_train = data_for_var.loc[data_for_var[variable].isna()==False,variable]

    X_test =  data_for_var.loc[data_for_var[variable].isna()==True,:]
    X_test = X_test.drop (variable, 1)


    classifier = RandomForestRegressor()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    X_test[variable] = y_pred

    data_out = data
    data_out[variable].fillna(X_test[variable], inplace = True)
    
    return data_out

In [ ]:
################################################ Create Binning ################################################

# columns = list(data_train)
# for variable in columns:
#     if data_train[variable].nunique() < 10 and variable != 'Survived':
#         print(data_train[[variable, 'Survived']].groupby([variable], as_index=False).mean().sort_values(by = variable, ascending = True))
#         print(' ')

#Harder decision for Titles
#data_train[['Title', 'Survived']].groupby(['Title'], as_index=False).count().sort_values(by = 'Survived', ascending = False)


# data_train['Fare_bands'] = pd.cut(data_train['Fare'], 10)
# data_train[['Fare_bands', 'Survived']].groupby(['Fare_bands'], as_index=False).mean().sort_values(by = 'Fare_bands', ascending = True)

# data_train['Age_bands'] = pd.cut(data_train['Age'], 10)
# data_train[['Age_bands', 'Survived']].groupby(['Age_bands'], as_index=False).mean().sort_values(by = 'Age_bands', ascending = True)

##################################################################################################################

In [15]:
def create_binning(data):
    
    #SibSp
    data.loc[data['SibSp'] == 0, 'SibSp_cat'] = '0'
    data.loc[(data['SibSp'] >= 1) & (data['SibSp'] <= 2) , 'SibSp_cat'] = '1-2'
    data.loc[(data['SibSp'] >= 3) & (data['SibSp'] <= 4) , 'SibSp_cat'] = '3-4'
    data.loc[data['SibSp'] > 4 , 'SibSp_cat'] = '5+'

    #Parch
    data.loc[data['Parch'] == 0, 'Parch_cat'] = '0'
    data.loc[(data['Parch'] >= 1) & (data['Parch'] <= 2) , 'Parch_cat'] = '1-2'
    data.loc[data['Parch'] == 3 , 'Parch_cat'] = '3'
    data.loc[data['Parch'] > 3 , 'Parch_cat'] = '4+'

    #FamilySize
    data.loc[data['FamilySize'] == 0, 'FamilySize_cat'] = '0'
    data.loc[data['FamilySize'] == 1, 'FamilySize_cat'] = '1'
    data.loc[(data['FamilySize'] >= 2) & (data['FamilySize'] <= 3) , 'FamilySize_cat'] = '2-3'
    data.loc[data['FamilySize'] == 4, 'FamilySize_cat'] = '4'
    data.loc[data['FamilySize'] > 4, 'FamilySize_cat'] = '5+'

    #Age
    data['Age_bands'] = pd.cut(data['Age'], 4)

    data.loc[(data['Age'] >= 0) & (data['Age'] <= 16) , 'Age_cat'] = '0-16'
    data.loc[(data['Age'] > 16) & (data['Age'] <= 50) , 'Age_cat'] = '16-50'
    data.loc[(data['Age'] > 50) & (data['Age'] <= 60) , 'Age_cat'] = '50-60'
    data.loc[data['Age'] > 60 , 'Age_cat'] = '60+'

    #Fare
    data['Fare_bands'] = pd.cut(data['Fare'], 10)

    data.loc[data['Fare'] <= 51 , 'Fare_cat'] = '0-51'
    data.loc[(data['Fare'] > 51) & (data['Fare'] <= 205) , 'Fare_cat'] = '51-205'
    data.loc[(data['Fare'] > 205) & (data['Fare'] <= 307) , 'Fare_cat'] = '205-307'
    data.loc[data['Fare'] > 307 , 'Fare_cat'] = '307+'
    
    data_out = data.drop(['SibSp', 'Parch', 'FamilySize', 'Age', 'Age_bands', 'Fare', 'Fare_bands'], axis = 1)
    
    return data_out

In [2]:
############################# Deleting variables #############################################################

#data_train = data_train.drop ('SibSp', 1)

In [3]:
######################################## Data Exporation #####################################################

####### Group by each categorical column, and check the response

# columns = list(data_before_dummies)
# for variable in columns:
#     if data_before_dummies[variable].nunique() < 10 and variable != 'Survived':
#         print(data_before_dummies[[variable, 'Survived']].groupby([variable], as_index=False).mean())
#         print(' ')

####### Look at your variables, think what might be correlated or requires special attention
#print ( *data_before_dummies.columns.tolist(), sep = "\n")

####### Heat map for variable correlations
#sns.heatmap(data_before_dummies.corr(), vmax=1., square=False, cmap="YlGnBu").xaxis.tick_top()

In [4]:
# Save the data 
#data_before_processing = data
#data_before_processing.to_csv('data_before_processing.csv', index = False, header = True)
#data_before_processing.head()

In [21]:
def create_new_var(data):
    
    import nbimporter
    from setup_code import get_title
    
    #family size
    data.loc[:,'FamilySize'] = data['SibSp'] + data['Parch'] + 1

    #is alone
    data.loc[:,'IsAlone'] = 0
    data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1    

    #title
    data.loc[:,'Title'] = data['Name'].apply(get_title)

    # Title
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Major', 
                                                  'Sir', 'Jonkheer', 'Dona'], 'Rare')

    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    
    ###
    data.loc[:,'Last_name'] = data['Name'].apply(get_last_name)
    data.loc[:,'Other_full_name'] = data['Name'].apply(get_other_full_name)
    data.loc[:,'Other_last_name'] = data['Name'].apply(get_other_last_name)
    data.loc[data['Cabin'].isnull() == False,'Cabin_letter'] = data.loc[data['Cabin'].isnull() == False,'Cabin'].apply(get_capital_letters)
    
    
    #Split for tickets
    
    #if there is a space, then split into two columns
    data.loc[data['Ticket'].str.find(' ') != -1, 'Ticket_first'] = data['Ticket'].str.split().str[0]
    data.loc[data['Ticket'].str.find(' ') != -1, 'Ticket_second'] = data['Ticket'].str.split().str[1]

    #if no space, assign it to the second
    data.loc[data['Ticket'].str.find(' ') == -1, 'Ticket_second'] = data['Ticket']


    return

In [26]:
def change_var_types(data):
    
    data_out = data
    data_out = data_out.copy() # explanation - https://www.dataquest.io/blog/settingwithcopywarning/
    
    data_out.loc[:,'Pclass'] = data_out['Pclass'].astype('object')
    data_out.loc[:,'IsAlone'] = data_out['IsAlone'].astype('object')
    
    data_out.loc[:,'SibSp'] = data_out['SibSp'].astype('object')
    data_out.loc[data_out['SibSp'].isin ([0,1,2,3,4]) == False , 'SibSp'] = '5+'

    data_out.loc[:,'Parch'] = data_out['Parch'].astype('object')
    data_out.loc[data_out['Parch'].isin ([0,1,2,3]) == False , 'Parch']  = '4+'
    
    data_out.loc[:,'FamilySize'] = data_out['FamilySize'].astype('object')
    data_out.loc[data_out['FamilySize'].isin ([0,1,2,3, 4]) == False , 'FamilySize']  = '5+'
    
    ###########
    
    data_out.loc[:,'Cabin_impact'] = data_out['Cabin_impact'].astype('float32')
    data_out.loc[:,'Ticket_impact'] = data_out['Ticket_impact'].astype('float32')
    data_out.loc[:,'Last_name_impact'] = data_out['Last_name_impact'].astype('float32')
    data_out.loc[:,'Other_last_name_impact'] = data_out['Other_last_name_impact'].astype('float32')  
    data_out.loc[:,'Ticket_second_impact'] = data_out['Ticket_second_impact'].astype('float32')
    data_out.loc[:,'Ticket_first_impact'] = data_out['Ticket_first_impact'].astype('float32')

    
    return data_out

In [8]:
#######################################  Imputing missing values ###############################################

#data_train.isna().sum()

# Survived, SibSp, Parch, IsAlone might have 0 and it makes sense

#check_for_missing = data_train.loc[:, data_train.columns.isin(['Survived', 'SibSp', 'Parch', 'IsAlone'])==False]
#check_for_missing.shape[0] - check_for_missing.fillna(0).astype(bool).sum(axis=0)

In [11]:
def add_missing_ind(data, variable, name):
    
    # Create a column for which ones values are missing
    data.loc[data[variable].isna()==True,name] = 1
    data.loc[data[variable].isna()==False,name] = 0

    # Make it categorical
    data.loc[:,name] = data[name].astype('object')
    
    return

In [12]:
def create_dummies(data, to_print):
    
    categorical_variables = data.select_dtypes(include = ['category', 'object']).columns.tolist()
    dummy_variables = pd.get_dummies(data.loc[:,categorical_variables])

    data = pd.merge(data, dummy_variables, left_index = True, right_index = True)
    data_out = data.drop(categorical_variables, axis = 1)
    
    if to_print == 1:
    
        print ("List of all categorical variables: ", "\n")
        print (*categorical_variables, sep = "\n")
        print (" ")

        print ("After creating dummy variables, the list looks like this: ", "\n")
        print ( *data_out.columns.tolist(), sep = "\n")
        print (" ")
        
    return data_out

In [13]:
# def impact_coding(data, variable, new_variable_name):
    
#     import math
#     from math import log
    
#     # WARNING: PLEASE DO NOT HAVE ANY NAs!!
#     data[variable].fillna('NA', inplace = True)

#     #variable = 'Embarked'
#     #new_variable_name = 'Embarked_impact'

#     response_variable = 'Survived'
#     distinct_values = data[variable].unique().tolist()

#     log_cols = [variable, new_variable_name]
#     log = pd.DataFrame(columns=log_cols)

#     for value in distinct_values:

#         subset = data.loc[data[variable] == value,:]
        
#         #probability of 1 for this level
#         level_summary = subset[[variable, response_variable]].groupby([response_variable], as_index=False).count()
        
#         level_ones = level_summary.loc[level_summary[response_variable] == 1, variable]
#         level_zeros = level_summary.loc[level_summary[response_variable] == 0, variable]

#         if (level_ones.empty is False) and (level_zeros.empty is False):
#             level_prob = level_ones.item()/(level_ones.item() + level_zeros.item())

#         elif (level_ones.empty is False):
#             level_prob = 1

#         elif (level_zeros.empty is False):
#             level_prob = 0
        
#         #probability of 1 for everything
#         overall_summary = data[[variable, response_variable]].groupby([response_variable], as_index=False).count()
        
#         overall_ones = overall_summary.loc[overall_summary[response_variable] == 1, variable]
#         overall_zeros = overall_summary.loc[overall_summary[response_variable] == 0, variable]
        
#         if (overall_ones.empty is False) and (overall_zeros.empty is False):
#             overall_prob = overall_ones.item()/(overall_ones.item() + overall_zeros.item())

#         elif (overall_ones.empty is False):
#             overall_prob = 1

#         elif (overall_zeros.empty is False):
#             overall_prob = 0
            
#         if level_prob!=0
#         impact = math.log(level_prob/(1-level_prob)) - math.log (overall_prob/(1-overall_prob))

#         log_entry = pd.DataFrame([[value, impact]], columns=log_cols)
#         log = log.append(log_entry)

#     data_out = pd.merge(data, log[[variable,new_variable_name]],  on=variable)
    
#     return data_out, log

In [19]:
def impact_coding(data, variable, new_variable_name):
    
    #source: https://github.com/WinVector/vtreat/blob/master/R/effectTreatmentC.R
    
    import numpy as np
    import scipy.special as spec
    
    epsilon = 1.0e-6
    smFactor = 1.0e-4

    # WARNING: PLEASE DO NOT HAVE ANY NAs!!
    data[variable].fillna('NA', inplace = True)

    #variable = 'Embarked'
    #new_variable_name = 'Embarked_impact'

    response_variable = 'Survived'
    distinct_values = data[variable].unique().tolist()

    log_cols = [variable, new_variable_name]
    log = pd.DataFrame(columns=log_cols)

    for value in distinct_values:

        subset = data.loc[data[variable] == value,:]
        
        level_summary = subset[[variable, response_variable]].groupby([response_variable], as_index=False).count()
        overall_summary = data[[variable, response_variable]].groupby([response_variable], as_index=False).count()
        
        true_cases = overall_summary.loc[overall_summary[response_variable] == 1, variable]
        false_cases = overall_summary.loc[overall_summary[response_variable] == 0, variable]

        if (true_cases.empty is False):
            nTrue = true_cases.item()
        else:
            nTrue = 0
            
        if (false_cases.empty is False):
            nFalse = false_cases.item()
        else:
            nFalse = 0
            
        true_cases_given_level = level_summary.loc[level_summary[response_variable] == 1, variable]
        false_cases_given_level = level_summary.loc[level_summary[response_variable] == 0, variable]
        
        if (true_cases_given_level.empty is False):
            nTrue_givenLevel = true_cases_given_level.item()
        else:
            nTrue_givenLevel = 0
            
        if (false_cases_given_level.empty is False):
            nFalse_givenLevel = false_cases_given_level.item()
        else:
            nFalse_givenLevel = 0
        
        #unconditional probability that target is true
        prior_probTrue = np.maximum(epsilon, np.minimum(1-epsilon, nTrue/(nTrue+nFalse)))
        
        #probability of a given evidence, condition on outcome = True
        prob_givenTrue = (nTrue_givenLevel + smFactor)/(nTrue + smFactor)
        
        #probability of a given evidence, condition on outcome = False
        prob_givenFalse = (nFalse_givenLevel + smFactor)/(nFalse + smFactor)
        
        #Bayes law, not normalized
        
        probTrue_givenLevel_unnorm = prior_probTrue * prob_givenTrue 
        probFalse_givenLevel_unnorm = (1-prior_probTrue) * prob_givenFalse 
        
        probTrue_givenLevel = probTrue_givenLevel_unnorm / (probTrue_givenLevel_unnorm + probFalse_givenLevel_unnorm)
        
        impact = spec.logit (probTrue_givenLevel) - spec.logit(prior_probTrue)

        log_entry = pd.DataFrame([[value, impact]], columns=log_cols)
        log = log.append(log_entry)

    data_out = pd.merge(data, log[[variable,new_variable_name]],  on=variable)
    
    return data_out, log